# Populariteit van een app

Een bedrijf heeft vorige maand een nieuwe app gelanceerd. Ze hebben iedere dag genoteerd hoeveel mensen deze gedownload hebben. Je vindt deze gegevens in “populariteitApp.csv”.

Er zijn in dit geval geen verkeerde of ontbrekende gegevens.  Je zou het bestand dus gemakkelijk moeten kunnen lezen. Je moet wel zorgen dat de kolom "downloads" van het type `float` wordt.


In [1]:
import pandas as pd

populariteitApp = pd.read_csv('data/populariteitApp.csv', sep=',')

display(populariteitApp)

,datum,downloads
0,2013-03-01,2
1,2013-03-02,5
2,2013-03-03,6
3,2013-03-04,4
4,2013-03-05,4
5,2013-03-06,9
6,2013-03-07,9
7,2013-03-08,10
8,2013-03-09,11
9,2013-03-10,13


1.	Maak een plot van de gegevens zodat je al wat inzicht krijgt in de data.

2.	Kan je de seizoensgrootte vinden?
Tip: gebruik acorr-functie op downloads die je eerst omzet naar Numpy array van floats.

3.	Voorspel nu, voor iedere voorspellingsmethode, de drie volgende dagen.  Bereken telkens de MAE, RMSE en de MAPE.

Vervolledig volgende tabel met alle resultaten.

|techniek               | dag 1 | dag 2 | dag 3 | MAE | RMSE | MAPE |
|-----------------------|-------|-------|-------|-----|------|------|
| naief                 | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|
| gemiddelde            | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|
| voortschrijdend(m=5)  | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|
| lin.comb. (m=13)      | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|
| trend                 | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|
| add. decomp. (m=7)    | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|
| multi.decomp. (m=7)   | &nbsp;| &nbsp;|&nbsp; |&nbsp;|&nbsp;|&nbsp;|

4.	lineaire combinatie: welke gewichten vind je?  Welke waarde speelt dus de grootste rol in het voorspellen van de volgende waarde?

5.	trend: wat is de formule van de regressielijn?

6.	additieve decompositie: wat is de formule voor de trendlijn?

7.	additieve decompositie: wat zijn de waarden voor het weerkerend patroon?

8.	multiplicatieve decompositie: wat is de formule voor de trendlijn?

9.	multiplicatieve decompositie: wat zijn de waarden voor het weerkerend patroon?

10.	welke techniek geeft de beste voorspelling en waarom?
